# PyTorch Training and Serving in SageMaker "Script Mode"

Script mode is a training script format for PyTorch that lets you execute any PyTorch training script in SageMaker with minimal modification. The [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) handles transferring your script to a SageMaker training instance. On the training instance, SageMaker's native PyTorch support sets up training-related environment variables and executes your training script. In this tutorial, we use the SageMaker Python SDK to launch a training job and deploy the trained model.

Script mode supports training with a Python script, a Python module, or a shell script. In this example, we use a Python script to train a classification model on the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). In this example, we will show how easily you can train a SageMaker using PyTorch scripts with SageMaker Python SDK. In addition, this notebook demonstrates how to perform real time inference with the [SageMaker PyTorch Serving container](https://github.com/aws/sagemaker-pytorch-serving-container). The PyTorch Serving container is the default inference method for script mode. For full documentation on deploying PyTorch models, please visit [here](https://github.com/aws/sagemaker-python-sdk/blob/master/doc/using_pytorch.rst#deploy-pytorch-models).

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

## Install SageMaker Python SDK

In [3]:
!pip install sagemaker --upgrade --ignore-installed --no-cache --user

    100% |████████████████████████████████| 296kB 22.2MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 44.3MB/s ta 0:00:01
    100% |████████████████████████████████| 20.2MB 83.6MB/s ta 0:00:01
    100% |████████████████████████████████| 1.3MB 102.2MB/s ta 0:00:01
    100% |████████████████████████████████| 26.1MB 95.1MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 64.2MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 65.2MB/s ta 0:00:01
    100% |████████████████████████████████| 5.9MB 85.3MB/s ta 0:00:01
    100% |████████████████████████████████| 583kB 93.7MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 96.8MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 95.4MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 98.5MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 67.4MB/s ta 0:00:01
    100% |████████████████████████████████| 552kB 95.9MB/s ta 0:00:01
    100% |██████████

In [4]:
!pip install torch==1.3.1 torchvision==0.4.2 --upgrade --ignore-installed --no-cache --user

    83% |██████████████████████████▋     | 612.0MB 80.4MB/s eta 0:00:024  12% |████                            | 91.4MB 79.5MB/s eta 0:00:09    12% |████▏                           | 95.4MB 80.9MB/s eta 0:00:08    14% |████▊                           | 107.5MB 81.5MB/s eta 0:00:08    23% |███████▋                        | 175.3MB 79.4MB/s eta 0:00:08    | 419.5MB 89.5MB/s eta 0:00:04

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



    91% |█████████████████████████████▏  | 670

    91% |█████████████████████████████▍  | 674.6MB 80.5MB/s eta 0:00:01

    92% |█████████████████████████████▋  | 678.8MB 82.9MB/s eta 0:00:01

...

    98% |███████████████████████████████▌| 723.6MB 76.4MB/s eta 0:00:01

    99% |███████████████████████████████▊| 727.6MB 88.4MB/s eta 0:00:01

    99% |███████████████████████████████▉| 731.6MB 85.0MB/s eta 0:00:01

    100% |████████████████████████████████| 734.6MB 88.4MB/s 


    100% |████████████████████████████████| 10.2MB 95.5MB/s ta 0:00:01
    100% |████████████████████████████████| 20.2MB 95.0MB/s ta 0:00:01                   | 563kB 68.1MB/s eta 0:00:01
    100% |████████████████████████████████| 2.1MB 91.9MB/s ta 0:00:01
docker-compose 1.24.1 has requirement requests!=2.11.0,!=2.12.2,!=2.18.0,<2.21,>=2.6.1, but you'll have requests 2.22.0 which is incompatible.
awscli 1.16.283 has requirement botocore==1.13.19, but you'll have botocore 1.13.49 which is incompatible.
awscli 1.16.283 has requirement rsa<=3.5.0,>=3.1.2, but you'll have rsa 4.0 which is incompatible.
  The scripts f2py, f2py3 and f2py3.6 are installed in '/home/ec2-user/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  The scripts convert-caffe2-to-onnx and convert-onnx-to-caffe2 are installed in '/home/ec2-user/.local/bin' which is not on PATH.
  Consider adding this directory to PATH

Forcing `pillow==6.2.1` due to https://discuss.pytorch.org/t/cannot-import-name-pillow-version-from-pil/66096

In [10]:
!pip uninstall -y pillow

Skipping pillow as it is not installed.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [11]:
!pip install pillow==6.2.1 --upgrade --ignore-installed --no-cache --user

    100% |████████████████████████████████| 2.1MB 94.2MB/s ta 0:00:01
docker-compose 1.24.1 has requirement requests!=2.11.0,!=2.12.2,!=2.18.0,<2.21,>=2.6.1, but you'll have requests 2.22.0 which is incompatible.
awscli 1.16.283 has requirement botocore==1.13.19, but you'll have botocore 1.13.49 which is incompatible.
awscli 1.16.283 has requirement rsa<=3.5.0,>=3.1.2, but you'll have rsa 4.0 which is incompatible.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Restart the Kernel to Recognize New Dependencies Above

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)

In [1]:
!pip3 list

Package              Version   
-------------------- ----------
absl-py              0.9.0     
astor                0.8.1     
astroid              1.6.6     
attrs                19.3.0    
awscli               1.16.76   
awscli-cwlogs        1.4.6     
backcall             0.1.0     
bleach               3.1.0     
bokeh                1.0.4     
boto                 2.49.0    
boto3                1.10.49   
botocore             1.13.49   
cachetools           4.0.0     
certifi              2019.11.28
chardet              3.0.4     
cloudpickle          1.2.2     
cmake                3.13.3    
colorama             0.3.9     
coremltools          2.0       
cpplint              1.3.0     
cycler               0.10.0    
dask                 2.6.0     
decorator            4.4.1     
defusedxml           0.6.0     
docutils             0.15.2    
entrypoints          0.3       
environment-kernels  1.1.1     
future               0.17.1    
gast                 0.2.2     
google-a

## Create the SageMaker Session

In [2]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

## Setup the Service Execution Role and Region
Get IAM role arn used to give training and hosting access to your data.  See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).

In [3]:
role = get_execution_role()
print('RoleARN:  {}\n'.format(role))

region = sagemaker_session.boto_session.region_name
print('Region:  {}'.format(region))

RoleARN:  arn:aws:iam::362377691630:role/service-role/AmazonSageMaker-ExecutionRole-20200109T002600

Region:  us-east-1


## Training Data

### Copy the Training Data to Your Notebook Disk

In [4]:
local_data_path = './data'

In [6]:
from torchvision import datasets, transforms

normalization_mean = 0.1307
normalization_std = 0.3081

# download the dataset
# this will not only download data to ./mnist folder, but also load and transform (normalize) them
datasets.MNIST(local_data_path, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((normalization_mean,), (normalization_std,))
]))

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [7]:
!ls -R {local_data_path}

./data:
MNIST

./data/MNIST:
processed  raw

./data/MNIST/processed:
test.pt  training.pt

./data/MNIST/raw:
t10k-images-idx3-ubyte	   train-images-idx3-ubyte
t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte	   train-labels-idx1-ubyte
t10k-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


### Upload the Data to S3 for Distributed Training Across Many Workers
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.

This is S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.

In [8]:
bucket = sagemaker_session.default_bucket()
data_prefix = 'sagemaker/pytorch-mnist/data'

In [9]:
training_data_uri = sagemaker_session.upload_data(path=local_data_path, bucket=bucket, key_prefix=data_prefix)
print('Input spec (S3 path): {}'.format(training_data_uri))

Input spec (S3 path): s3://sagemaker-us-east-1-362377691630/sagemaker/pytorch-mnist/data


In [10]:
!aws s3 ls --recursive {training_data_uri}

2020-01-09 05:51:34    7920466 sagemaker/pytorch-mnist/data/MNIST/processed/test.pt
2020-01-09 05:51:34   47520466 sagemaker/pytorch-mnist/data/MNIST/processed/training.pt
2020-01-09 05:51:34    7840016 sagemaker/pytorch-mnist/data/MNIST/raw/t10k-images-idx3-ubyte
2020-01-09 05:51:33    1648877 sagemaker/pytorch-mnist/data/MNIST/raw/t10k-images-idx3-ubyte.gz
2020-01-09 05:51:34      10008 sagemaker/pytorch-mnist/data/MNIST/raw/t10k-labels-idx1-ubyte
2020-01-09 05:51:33       4542 sagemaker/pytorch-mnist/data/MNIST/raw/t10k-labels-idx1-ubyte.gz
2020-01-09 05:51:33   47040016 sagemaker/pytorch-mnist/data/MNIST/raw/train-images-idx3-ubyte
2020-01-09 05:51:34    9912422 sagemaker/pytorch-mnist/data/MNIST/raw/train-images-idx3-ubyte.gz
2020-01-09 05:51:33      60008 sagemaker/pytorch-mnist/data/MNIST/raw/train-labels-idx1-ubyte
2020-01-09 05:51:34      28881 sagemaker/pytorch-mnist/data/MNIST/raw/train-labels-idx1-ubyte.gz


## Train
### Training Script
The `pytorch_mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [11]:
!ls ./src/mnist_pytorch.py

./src/mnist_pytorch.py


You can add custom Python modules to the `src/requirements.txt` file.  They will automatically be installed - and made available to your training script.

In [12]:
!cat ./src/requirements.txt

# Python dependencies go here

### Train with SageMaker `PyTorch` Estimator

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure.  We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters.  In this case we are going to run our training job on two(2) `ml.p3.2xlarge` instances.  Alternatively, you can specify `ml.c4.xlarge` instances.  This example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)).  The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist.py` script above.

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem of each worker, so our training script can simply read the data from disk.

### `fit` the Model (Approx. 15 mins)

To start a training job, we call `estimator.fit(training_data_uri)`.

In [13]:
from sagemaker.pytorch import PyTorch
import time

model_output_path = 's3://{}/sagemaker/pytorch-mnist/training-runs'.format(bucket)

mnist_estimator = PyTorch(
                  entry_point='mnist_pytorch.py',
                  source_dir='./src',
                  output_path=model_output_path,
                  role=role,
                  framework_version='1.3.1',
                  train_instance_count=1,
                  train_instance_type='ml.c5.2xlarge',
                  hyperparameters={
                    'epochs': 5,
                    'backend': 'gloo'
                  },
                  # Assuming the logline from the PyTorch training job is as follows:
                  #    Test set: Average loss: 0.3230, Accuracy: 9103/10000 (91%)
                  metric_definitions=[
                     {'Name':'test:loss', 'Regex':'Test set: Average loss: (.*?),'},
                     {'Name':'test:accuracy', 'Regex':'(.*?)%;'}
                  ]
)

mnist_estimator.fit(inputs={'training': training_data_uri},
                                        wait=False)

training_job_name = mnist_estimator.latest_training_job.name

print('training_job_name:  {}'.format(training_job_name))

training_job_name:  pytorch-training-2020-01-09-05-51-37-112


Attach to a training job to monitor the logs.

_Note:  Each instance in the training job (2 in this example) will appear as a different color in the logs.  1 color per instance._

In [14]:
mnist_estimator = PyTorch.attach(training_job_name=training_job_name)

2020-01-09 05:51:37 Starting - Starting the training job...
2020-01-09 05:51:38 Starting - Launching requested ML instances......
2020-01-09 05:52:44 Starting - Preparing the instances for training...
2020-01-09 05:53:26 Downloading - Downloading input data...
2020-01-09 05:54:06 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2020-01-09 05:54:07,354 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2020-01-09 05:54:07,356 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-01-09 05:54:07,365 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2020-01-09 05:54:08,814 sagemaker_pytorch_container.training INFO     Invoking user training script.
2020-01-09 05:54:09,080 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 

INFO:__main__:Train Epoch: 2 [25600/60000 (43%)] Loss: 0.663549
Train Epoch: 1 [51200/60000 (85%)] Loss: 0.573531
INFO:__main__:Train Epoch: 2 [32000/60000 (53%)] Loss: 0.588324
Train Epoch: 1 [57600/60000 (96%)] Loss: 0.427717
INFO:__main__:Train Epoch: 2 [38400/60000 (64%)] Loss: 0.416387
Test set: Average loss: 0.2076, Accuracy: 9400/10000 (94%)
INFO:__main__:Train Epoch: 2 [44800/60000 (75%)] Loss: 0.295054

INFO:__main__:Train Epoch: 2 [51200/60000 (85%)] Loss: 0.536017
Train Epoch: 2 [6400/60000 (11%)] Loss: 0.741295
INFO:__main__:Train Epoch: 2 [57600/60000 (96%)] Loss: 0.298408
Train Epoch: 2 [12800/60000 (21%)] Loss: 0.407144
INFO:__main__:Test set: Average loss: 0.1338, Accuracy: 9594/10000 (96%)
Train Epoch: 2 [19200/60000 (32%)] Loss: 0.381953

Train Epoch: 2 [25600/60000 (43%)] Loss: 0.663549
INFO:__main__:Train Epoch: 3 [6400/60000 (11%)] Loss: 0.398564
Train Epoch: 2 [32000/60000 (53%)] Loss: 0.588324
INFO:__main__:Train Epoch: 3 [12800/60000 (21%)] Loss: 0.230535
Train 

## Option 1:  Perform Batch Predictions Directly in the Notebook

Use PyTorch Core to load the model from `model_output_path`

In [15]:
!aws --region {region} s3 ls --recursive {model_output_path}/{training_job_name}/output/

2020-01-09 05:55:31      82012 sagemaker/pytorch-mnist/training-runs/pytorch-training-2020-01-09-05-51-37-112/output/model.tar.gz


In [16]:
!aws --region {region} s3 cp {model_output_path}/{training_job_name}/output/model.tar.gz ./model/model.tar.gz

download: s3://sagemaker-us-east-1-362377691630/sagemaker/pytorch-mnist/training-runs/pytorch-training-2020-01-09-05-51-37-112/output/model.tar.gz to model/model.tar.gz


In [17]:
!ls ./model

model.tar.gz


In [18]:
!tar -xzvf ./model/model.tar.gz -C ./model

model.pth


In [19]:
# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [20]:
import torch

loaded_model = Net().to('cpu')
# single-machine multi-gpu case or single-machine or multi-machine cpu case
loaded_model = torch.nn.DataParallel(loaded_model)
print(loaded_model)

DataParallel(
  (module): Net(
    (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
    (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
    (conv2_drop): Dropout2d(p=0.5, inplace=False)
    (fc1): Linear(in_features=320, out_features=50, bias=True)
    (fc2): Linear(in_features=50, out_features=10, bias=True)
  )
)


In [21]:
loaded_model.load_state_dict(torch.load('./model/model.pth', map_location='cpu'))

<All keys matched successfully>

In [22]:
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=256, 
    shuffle=True
)

single_loaded_img = test_loader.dataset.data[0]
single_loaded_img = single_loaded_img.to('cpu')
single_loaded_img = single_loaded_img[None, None]
single_loaded_img = single_loaded_img.type('torch.FloatTensor') # instead of DoubleTensor

print(single_loaded_img.numpy())

[[[[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
      0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
      0.   0.]
   [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
      0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
      0.   0.]
   [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
      0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
      0.   0.]
   [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
      0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
      0.   0.]
   [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
      0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
      0.   0.]
   [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
      0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
      0.   0.]
   [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
      0.   0.

In [23]:
from matplotlib import pyplot as plt

plt.imshow(single_loaded_img.numpy().reshape(28, 28), cmap='Greys')

In [24]:
result = loaded_model(single_loaded_img)
prediction = result.max(1, keepdim=True)[1][0][0].numpy()
print(prediction)

7


## Option 2:  Create a SageMaker Endpoint and Perform REST-based Predictions

### Deploy the Trained Model to a SageMaker Endpoint (Approx. 10 mins)

After training, we use the `PyTorch` estimator object to build and deploy a `PyTorchPredictor`. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `pytorch_mnist.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers).

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `mnist.py`.

In [ ]:
predictor = mnist_estimator.deploy(initial_instance_count=1, instance_type='ml.c5.2xlarge')

-------------

### Invoke the Endpoint

We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a `data` variable in this notebook, which we can then pass to the `predictor`.

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

The value of `data` is retrieved from the HTML above.

In [ ]:
print(data)

In [ ]:
import numpy as np

image = np.array([data], dtype=np.float32)
response = predictor.predict(image)
prediction = response.argmax(axis=1)[0]
print(prediction)

### (Optional) Cleanup Endpoint

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)